# Step-back Prompting

1. ユーザーの元の質問に基づいて、ステップバック質問を生成
2. 元の質問とステップバック質問の両方を情報収集
3. 取得した両方の情報に基づいて回答を生成

In [11]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "警察官は合法的な逮捕を行うことができますか？",
        "output": "警察官は何ができるのですか？",
    },
    {
        "input": "Jan Sindelはどこの国で生まれましたか？",
        "output": "Jan Sindelの個人史は何ですか？",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': '警察官は合法的な逮捕を行うことができますか？', 'output': '警察官は何ができるのですか？'}, {'input': 'Jan Sindelはどこの国で生まれましたか？', 'output': 'Jan Sindelの個人史は何ですか？'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは世界の知識のエキスパートです。あなたの仕事は、一歩下がって、より答えやすい一般的な一歩下がった質問に言い換えることです。いくつか例を挙げましょう："),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='あなたは世界の知識のエキスパートです。あなたの仕事は、一歩下がって、より答えやすい一般的な一歩下がった質問に言い換えることです。いくつか例を挙げましょう：'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '警察官は合法的な逮捕を行うことができますか？', 'output': '警察官は何ができるのですか？'}, {'input': 'Jan Sindelはどこの国で生まれましたか？', 'output': 'Jan Sindelの個人史は何ですか？'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])), Huma

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


generate_queries_step_back = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0) | StrOutputParser()
question = "LLMエージェントにおけるタスク分解とは何ですか？"
generate_queries_step_back.invoke({"question": question})

'タスク分解とは何ですか？'

In [15]:
from langchain_core.runnables import RunnableLambda

response_prompt_template = """あなたは世界を知るエキスパートです。
私はあなたに質問します。
あなたの回答は包括的であるべきであり、以下の文脈が関連するのであれば、
それらと矛盾しないものでなければならない。
そうでなければ、関連性がなければ無視してください。

# {normal_context}
# {step_back_context}

# オリジナルの質問: {question}
# 回答:"""

response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # invokeで文字列を渡すのではなく {"question": "質問"} というdictで渡しているため整形が必要
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"]
    }
    | response_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

question = "RAGとはなんですか？"
print(chain.invoke({"question": question}))

RAG（Retrieval-Augmented Generation）とは、ファイルを参照して回答できる大規模言語モデル（LLM）を作成するための手法です。RAGの主な目的は、通常のLLMが持つ限界を克服することにあります。具体的には、通常のLLMは事実と異なる情報を生成する「ハルシネーション」や、学習データに含まれていない情報に対して正確な回答ができないという問題があります。

RAGの仕組みはシンプルで、ユーザーが質問をすると、まず外部データベースから関連するドキュメントを取得し、そのドキュメントとユーザーの質問をセットにしてLLMに渡します。LLMはこの情報を基に回答を生成します。このプロセスにより、より正確で関連性の高い回答を得ることが可能になります。

RAGを利用することで、情報が不十分な場合には回答をしない選択ができたり、独自のドキュメントに基づいて回答することができるため、実務においても非常に有用です。
